In [1]:
import pandas as pd
import numpy as np
import torch
from collections import Counter
import random
import scipy.stats
from sklearn.preprocessing import MinMaxScaler
import joblib
import torchvision.transforms as transforms
import albumentations as A

In [108]:
"""labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - femur non-standard plane
4 - femur standard plane
5 - femur non standard plane
6 - femur standard plane"""

'labels:\n0 - other\n1 - head non-standard plane\n2 - head standard plane\n3 - femur non-standard plane\n4 - femur standard plane\n5 - femur non standard plane\n6 - femur standard plane'

In [109]:
train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_train_scaled_size.csv')
val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_val_scaled_size.csv')
test = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_test_scaled_size.csv')

In [11]:
train_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_train.csv')
test_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_test.csv')

val_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_val.csv')

In [12]:
train['days'] = train_org['days']
test['days'] = test_org['days']
val['days'] = val_org['days']

In [14]:
train = pd.concat([train, test])

In [16]:
train.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/class_data/biometry_train_all.csv', index=False)
val.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/class_data/biometry_val_all.csv', index=False)

In [124]:
train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_train_scaled_size_all.csv')
val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_val_scaled_size_all.csv')

In [122]:
long_vids = (train['video'].value_counts() > 3)
videos = train['video'].unique()
long_vids = videos[long_vids]
train_filtered = train[train['video'].isin(long_vids)]

In [213]:
data = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_train_scaled_size_all.csv')

In [ ]:
pd.DataFrame({'video': data['video'].unique()}).to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/videos_train.csv', index=False)

In [236]:
data_val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_val_scaled_size_all.csv')

In [240]:
data_val['video'].unique()

array(['9_2', '19_2', '25_2', '30_2', '38_2', '77_2', '88_2', '105_2',
       '129_2', '169_2', '222_2', '237_2', '243_2', '245_2', '272_2',
       '294_2', '295_2', '308_2', '322_2', '343_2', '352_2', '410_2',
       '436_2', '447_2', '474_2', '489_2', '594_2', '2_3', '83_3', '38_3',
       '272_3', '294_3', '295_3', '343_3', '376_3', '378_3', '409_3',
       '453_3', '474_3', '521_3', '5_3', '20_3', '33_3', '39_3', '50_3',
       '56_3', '68_3', '87_3', '90_3', '116_3', '123_3', '148_3', '168_3',
       '171_3', '180_3', '182_3', '198_3', '199_3', '206_3', '218_3',
       '249_3', '259_3', '276_3', '279_3', '315_3', '326_3', '330_3',
       '332_3', '342_3', '354_3', '359_3', '367_3', '377_3', '383_3',
       '385_3', '386_3', '401_3', '420_3', '426_3', '462_3', '472_3',
       '485_3', '500_3', '525_3', '526_3', '533_3', '584_3', '587_3',
       '590_3', '598_3', '600_3', '604_3', '611_3', '636_3', '653_3',
       '670_3', '672_3', '680_3', '2_1', '4_1', '15_1', '45_1', '49_1',
    

In [241]:
pd.DataFrame({'video': data_val['video'].unique()}).to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/videos_val.csv', index=False)

In [225]:
data

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
0,6_2_35,4,6_2,16.47,0.120130,51,0.618863,147,0.686275,86.49360,115.32480,720,960,0.457089
1,6_2_36,4,6_2,16.47,0.120130,51,0.618863,147,0.705882,86.49360,115.32480,720,960,0.457089
2,6_2_37,4,6_2,16.47,0.120130,51,0.618863,147,0.725490,86.49360,115.32480,720,960,0.457089
3,6_2_38,4,6_2,16.47,0.120130,51,0.618863,147,0.745098,86.49360,115.32480,720,960,0.457089
4,6_2_39,4,6_2,16.47,0.120130,51,0.618863,147,0.764706,86.49360,115.32480,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36250,117_1_40,2,117_1,19.85,0.108639,44,0.841410,161,0.909091,78.22008,104.29344,720,960,0.563345
36251,117_1_41,2,117_1,19.85,0.108639,44,0.841410,161,0.931818,78.22008,104.29344,720,960,0.563345
36252,117_1_42,2,117_1,19.85,0.108639,44,0.841410,161,0.954545,78.22008,104.29344,720,960,0.563345
36253,117_1_43,2,117_1,19.85,0.108639,44,0.841410,161,0.977273,78.22008,104.29344,720,960,0.563345


In [226]:
index = '6_2_35'

In [234]:
idb = data.query('index == @index').iloc[0]

In [235]:
idb[0]

'6_2_35'

In [125]:
long_vids = (val['video'].value_counts() > 3)
videos = val['video'].unique()
long_vids = videos[long_vids]
val_filtered = val[val['video'].isin(long_vids)]

In [127]:
val_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/key_frame_data/biometry_val_all_filtered.csv', index=False)

In [130]:
def clamp(num, min_value, max_value):
   return max(min(num, max_value), min_value)

In [ ]:
class Video_Loader():
    def __init__(self, root, videos_path, ann_path, transform=None, target_transform=None, img_scaling = False):
        
        self.data_path = root
        self.ann_path = ann_path
        self.videos = pd.read_csv(videos_path)
        self.data = pd.read_csv(self.ann_path)
        self.img_scaling = img_scaling
        
        
    def _load_image(self, path):
        try:
            im = Image.open(path)
            im.convert('RGB')
        except:
            print("ERROR IMG LOADED: ", path)
            random_img = np.random.rand(224, 224, 3) * 255
            im = Image.fromarray(np.uint8(random_img))
        return im
    
    def __len__(self):
        return len(self.data)
    
    
    def _load_item(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        idb = self.data[data['index'] == index]
        frame_idx = idb[0]
        Class = torch.tensor(idb[1])
        video = idb[2]
        measure = torch.tensor(idb[3])
        ps = torch.tensor(idb[4])
        frames_n = torch.tensor(idb[5])
        measure_scaled = torch.tensor(idb[6], dtype=torch.float32)
        days_normalized = torch.tensor(idb[7], dtype=torch.float32)
        frame_loc = torch.tensor(idb[8], dtype=torch.float32)
        height = torch.tensor(idb[9])
        width = torch.tensor(idb[10])
        measure_normalized = torch.tensor(idb[13], dtype=torch.float32)
        images = self._load_image(self.data_path  + frame_idx + '.png')
        
        new_height = clamp(int(height*2.161191086437513), 0, 512)
        new_width = clamp(int(width*2.161191086437513), 0, 512)
        if self.img_scaling:
            padding = A.PadIfNeeded(min_height=512, min_width=512, border_mode=0, value=0, mask_value=0, always_apply=False, p=1.0)
            rescale = A.Resize(new_height, new_width)
            images = rescale(image = images)['image']
            images = padding(image=images)['image']
            images = np.expand_dims(images, 2)
            t = transforms.Compose([transforms.ToTensor(),])
            images = t(images)
        else:
            images = np.expand_dims(images, 2)
            t = transforms.Compose([transforms.ToTensor(),
            transforms.Resize((450, 600)),
            transforms.Pad((0, 0, 0, 150), fill = 0, padding_mode = 'constant'),
            transforms.Resize((512, 512)),
            transforms.Normalize(mean=0.1354949, std=0.18222201)])
            images = t(images)
        if self.transform is not None:
            images = self.transform(images)

        return images, Class, measure, ps, frames_n, measure_scaled, index, days_normalized, frame_loc, measure_normalized
    
    
    def __getitem__(self, index):
        video = self.videos[index]
        batch = self.data[self.data['video'] == video].sample(8)
        frame_idx = batch['index']
        images = list()
        Classes = list()
        indexes = list()
        for i in frame_idx:
            image, Class, measure, ps, frame_n, measure_scaled, index, days_normalized, frame_loc, measure_normalized = self._load_item(i)
            images.append(image)
            Classes.append(Class)
            indexes.append(index)
        return images, Classes, measure_scaled, ps, frame_n, measure_scaled, index, days_normalized, frame_loc, measure_normalized

In [18]:
def features_padding(features, max_length, split_sizes):
    padded_output = []
    for i, feature in enumerate(features):
        padding_length = max_length - split_sizes[i]
        padded_seq = torch.nn.functional.pad(feature, (0, padding_length, 0, 0, 0, 0), mode='constant', value=0)
        padded_output.append(padded_seq)
    padded_output = torch.stack(padded_output)
    return padded_output

In [30]:
original_seq = torch.ones(1, 1280, 13) # org lens: 3, 2, 4, 4
split_sizes = [3, 2, 4, 4]
tensor_list2 = [original_seq[:, :, i:i+split_size] for i, split_size in enumerate(split_sizes)]

In [31]:
tensor_list = torch.split(original_seq, split_size_or_sections = split_sizes, dim=2)

In [39]:
(list(tensor_list) == tensor_list2).all()

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [49]:
k

NameError: name 'k' is not defined

In [20]:
padded_seq = features_padding(tensor_list, 8, split_sizes)

In [370]:
padded_seq.shape

torch.Size([4, 1, 1280, 8])

In [350]:
import torch

def mask_sequence(padded_seq, org_seq_lens):  
    
    # Define the padded sequence
    padded_seq_len = padded_seq.size(-1)
    batch_len = padded_seq.size(0)
    # Define the mask tensor
    mask = torch.zeros((batch_len, padded_seq_len, padded_seq_len), dtype=torch.float32)
    
    # Set the non-padding elements to 1's
    for i in range(batch_len):
        mask[i, :, :org_seq_lens[i]] = 1
    
    
    return mask




padded_seq.shape

In [351]:
mask = mask_sequence(padded_seq, split_sizes)

In [357]:
padded_seq = padded_seq.squeeze(1)

In [358]:
padded_seq.shape

torch.Size([4, 1280, 8])

In [359]:
softmax = torch.nn.Softmax(dim = 2)
keys = padded_seq*torch.ones(4, 1280, 8)
queries = padded_seq*torch.ones(4, 1280, 8)
values = padded_seq*torch.ones(4, 1280, 8)

matmul = torch.matmul(queries.permute(0, 2, 1), keys) # (batch_size, n_channels, n_frames)
matmul.shape

torch.Size([4, 8, 8])

In [360]:
if mask is not None:
    matmul_masked = matmul.masked_fill(mask == 0, -1e20)
    print('matmul masked', matmul_masked)


matmul masked tensor([[[ 1.2800e+03,  1.2800e+03,  1.2800e+03, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, -1.0000e+20],
         [ 1.2800e+03,  1.2800e+03,  1.2800e+03, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, -1.0000e+20],
         [ 1.2800e+03,  1.2800e+03,  1.2800e+03, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, -1.0000e+20],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, -1.0000e+20],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, -1.0000e+20],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, -1.0000e+20],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, -1.0000e+20],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+20, -1.0000e+20,
          -1.0000e+20, -1.0000e+20, 

In [361]:
matmul_masked.shape

torch.Size([4, 8, 8])

In [362]:
softmax_output = softmax(matmul_masked) # (batch_size, n_channels, n_frames)
print('softmax', softmax_output)


softmax tensor([[[0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.500

In [363]:
attention_map = torch.matmul(values, softmax_output) # (batch_size, n_channels, n_frames)
print('attention_map', attention_map)
print('attention_map', attention_map.shape)
#print('attention_map', attention_map)

attention_map tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]],

        [[1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1., 

In [365]:
padded_seq[:, :3, :].shape

torch.Size([4, 3, 8])

In [34]:
random.randint(0, 0)

0

In [3]:
test = torch.rand(5, 1280, 8)

In [4]:
for i in test:
    print(i.shape)

torch.Size([1280, 8])
torch.Size([1280, 8])
torch.Size([1280, 8])
torch.Size([1280, 8])
torch.Size([1280, 8])


In [12]:
split_sizes = [3, 2, 4, 4]
org_batch = [torch.rand(i, 1, 512, 512) for i in split_sizes]
test_batch = torch.cat(org_batch)


In [13]:
test_batch_unbinded = torch.split(test_batch, split_size_or_sections= split_sizes, dim=0)

In [15]:
features = torch.rand(1, 1280, 13)

In [17]:
padded_seq = features_padding(features, 8, split_sizes)

RuntimeError: Padding length too large

In [9]:
len(test_batch_unbinded[3])

4

In [10]:
assert len(test_batch_unbinded) == len(split_sizes)

In [11]:
assert (test_batch_unbinded[2] == org_batch[2]).all()

In [26]:
len(org_batch)

4